# Score Aggregation Methods

We want to rank answers so that the best content floats to the top. The information we get is users' ratings of the answers. We must aggregate users' ratings in two ways: 

1. A single user will rate multiple aspects of a single answer (within-rating aggregation)
2. Multiple users rate a given answer (between-rating aggregation)

I'm focusing on the second kind of aggregation. Currently, SWARM just averages the scores.

## Rating honestly vs. strategically

We've noticed that as the deadline approaches, people are giving items more extreme scores (closer to 0 or 100) in order to move the average further, and get the item to the top or bottom of the pile. This is a sensible voting strategy. If I see that an item has a current score of 70 but believe it should have a score of 85, then I should give it a rating of 100 rather than 85, to move it closer to the correct score.

However, this voting strategy causes problems. It means our data on ratings is next to useless - we don't learn users' true opinion of the answer quality. It's also undemocratic. A user who gives an extreme rating has more leverage than a user who gives a rating close to the current average. 

Ideally, SWARM would use an aggregation method that elicits users' honest opinions of the answers, giving helpful feedback to the answer author and helpful data to the analytics team. Ideally we would rank items in a way that doesn't privilege one rater over another.

## Rank aggregation & impossibility theorems

To remove the incentive to gve extreme scores, the obvious solution is to use a rank aggregation method. Each user rates several answers, giving each a score out of 100 (after within-rating aggregation), but we don't need to use the exact numbers. Each rater's list of scores can be coarsened into a rank order of preferences. We can then use a standard method for aggregating ranks rather than real-valued scores.

Unfortunately, the [Gibbard-Satterthwaite Theorem](https://en.wikipedia.org/wiki/Gibbard%E2%80%93Satterthwaite_theorem) shows that rank aggregation methods cannot satisfy our desiderata: all "strategy-proof" methods for ranking more than two options must give all the power to a single rater. 

This might be particularly problematic for SWARM. Many voting systems are _robust_ to strategic voting, because the strategic voter needs lots of information about other voters' preferences. SWARM doesn't tell you exactly how other people are rating, but it lets you _change your own rating_ an arbitrary number of times. Users who wish to vote strategically could try different ratings until they achieve their desired outcome. 

[The space of possible rankings is large](https://en.wikipedia.org/wiki/Ordered_Bell_number), so if there are five or more answers, users cannot explore the space of rankings in a reasonable time. Even exploring a subset would take much more effort than just giving an extreme score. But it is an open question how often users would vote strategically if we aggregated answers like this.

The Gibbard-Satterthwaite Theorem tells us there is no perfect system, but we may be able to find a decent option - a system that produces something close to the ideal ranking in most cases, and which can't be manipulated without spending a lot of time and effort.

## Evaluating performance

The "ideal ranking" is undefined for any of our real examples, but we have some intuitions about which answers are best. I propose testing each rank aggregation method by looking at how it ranks the answers to **past questions** on SWARM Beta. A good method will accord pretty well with our intuitions; a bad one will not. However, this evaluation only gives us information about a small number of cases, and doesn't tell us about robustness to strategies other than "give extreme scores".

We can also evaluate performance using **synthetic data**. We randomly generate answers of varying quality, and sets of ratings that are realistically sparse and noisy. We apply the rank aggregation methods to this data, and see how close the overal ranking comes to the true rank order of the answers based on their quality. 

Because we can sample repeatedly, evaluation on synthetic data tells us about the average case performance of the ranking methods. However, it doesn't tell us about robustness to particular strategies (unless we build those strategies into the rating generation method, which would take a lot of work).

## Ranking Methods

I implemented and tested the following ranking methods:

1. Rank of average score (status quo in SWARM)
2. RdR's ranking method ("place everything with more than one rating above everthing with one rating. Within the two groups, rank by average score")
3. [Borda count](https://en.wikipedia.org/wiki/Borda_count) ("For each rater, for each answer, count how many other answers it _outranks_. Then for each answer, sum this count across all raters")
4. User Preference Ranking. This uses the exact score rather than just the rank, but it aggregates pairwise comparisons of items, among the set of users who have rated both items in the pair. It is more robust to sparse ranking. However it is still vulnerable to user manipultion using extreme scores. User Preference Ranking is described in Chapter 10 of _Who's #1?_ by Langville & Meyer.
5. Anca's ranking method: essentially User Preference Ranking, but instead of the exact point difference between two answers, I just ask which one "won" according to that rater. The proportion of wins vs losses, among raters who rated both answers, is the pairwise difference between those two answers. Because it only considers rank differences, Anca's method is not vulnerable to manipulation by giving extreme scores.
6. Median score
7. Geometric mean
8. Sum
8. Borda, normalized so that every rater's ratings add to 1 ("Tim ranking")
9. Borda, normalized so that every rater's ratings has maximum 1 ("Tim2 ranking")
10. Lizzie ranking: test whether Anca ranking correlates with number of ratings per answer. If the correlation is non-significant or negative, return Anca ranknig vector. If significant and positive, add some weight for the number of rankings the answer received, where the weight is proportional to the size of the correlation.

All these methods start by creating a rater-by-answer matrix of ratings.

### Simulation study

I started by generating synthetic data. I varied the following parameters:

* ```rnoise```, a scaling factor that increases the amount of noise in the ratings
* ```punrated```, the probability that a given rating will be hidden
* ```pslacker```, the probability that a user will fail to submit any ratings 
* ```plurker```, the probability that a user will fail to submit an answer
* ```toprated```: if this is True, the user will rate the answers they consider best, and fail to rate the answers they consider worse. 
    * Note: There is no set number of ratings the user completes. The number of ratings for each user, *k*, is drawn from a binomial with probability = ```1 - punrated```. Whether those *k* ratings are then distributed uniformly, or the user rates their *k* top answers, depends on ```toprated```.
* ```enoise```: Each user has an "easiness" bias parameter that is drawn from a Gaussian, reflecting that some users are harsher raters and some are easier. ```enoise``` is the standard deviation of that Gaussian.

### Results

I found that ```toprated``` had a big influence on which methods worked best. When ```toprated``` was True, the Borda count and related methods all did best. When it was False, all the other methods did about equally well, and the Borda count methods did much worse.

I believe this is because Borda count essentially awards a bonus for each rating. Each rating can only improve an item's Borda count. By contrast, using mean_rating(), an additional rating can *hurt* an item's mean score; likewise with User Preferences, Anca Rating, etc. If users are only rating the best items, then "receiving lots of ratings" is a meaningful signal that should be reflected in the score. However if users rate a random selection of items, then "receiving lots of ratings" means nothing and should not be rewarded.

The only method that performed pretty well in both conditions was Lizzie ranking. It does well because it tries to *detect* whether people are rating the top answers or not, and adjusts the score accordingly. Lizzie Rating didn't do as well as Borda count when ```toprated``` was True, but it did better than any non-Borda method, and when ```toprated``` was False it far outperformed Borda count. On the basis of these results I selected Lizzie ranking as the method to use. 

### Appendix: Methods I did NOT implement:

Evan Miller's [confidence interval-based method](http://www.evanmiller.org/how-not-to-sort-by-average-rating.html), the [Bayesian version](http://www.evanmiller.org/bayesian-average-ratings.html), the [star-rating version](http://www.evanmiller.org/ranking-items-with-star-ratings.html), and [Paul Masurel's variation](https://fulmicoton.com/posts/bayesian_rating/) take the _uncertainty_ around the average score into account. They rank by the lower bound of a confidence interval or credible interval around the average score, or smooth the average using a pseudocount. Unfortunately these methods are still centered on the mean, so they would still be vulnerable to the strategy of "give extreme scores". Users who give extremely low scores would have more power than before. Furthermore, we are unlikely to ever get enough ratings for the intervals to become narrow.

Minimum Violation Ranking (MVR), as described in Chapter 15 of _Who's #1?_ by Langville & Meyer, would take a set of users' rankings and produce an aggregate ranking that disagrees as little as possible with the input rankings. This method is optimal among all methods that use only rank data (as opposed to, for example, "average score" or "user preference rankings" which use the exact scores rather than just the ranks). It would do as well as or better than Borda count. However, the exact version of MVR requires solving an integer linear program  every time the scores are updated, which can take a long time. Langville & Meyer show you can speed it up by first solving a relaxed version of the problem. 

I chose not to implement MVR because (a) the implementation would be moderately complicated, (b) the runtime might be prohibitive, and (c) it's designed for aggregating complete rankings, not partial rankings. We would first turn the partial rankings into complete rankings by placing all unranked items _last_ in each user's ranking. So if I haven't had time to rate an answer, that is indistinguishable from me saying it is the worst answer. Given how sparse our data is, this is a very common case, so I think MVR would work poorly on our data.


## Data

For evaluating the methods on synthetic data, I have developed a class that pulls the data from a CSV of a synthetic rater-by-answer matrix.


In [1]:
# # need this on my laptop because my python & anaconda installation is all kinds of messed up
# import sys
# sys.path.append("/Users/lizzie/anaconda/")
# sys.path.append("/Users/lizzie/Library/Python/3.6/lib/python/site-packages")


## Define abstract score matrix class

This class implements the ranking methods. It's abstract because the _data_ for a score matrix could either be pulled from the SWARM DB, or generated synthetically, and I want to separate those cases. Those concrete classes are defined below.

In [285]:
import numpy as np
import scipy.stats as stats
from abc import ABCMeta, abstractmethod

class AbstractScoreMatrix(object):
    """A rater-by-answer matrix of scores for answers to the question.
    
    Includes several ranking methods:
        (e) anca_rating()
        (j) lizzie_rating()
        
    All of them return a 1D Numpy array with the *scores* of the answers.
    
    To implement these methods you need a user-by-answer matrix, where each cell contains a  
    single number representing that user's rating of that answer. Ratings are between 0 and 100. 
    
    If the user has not rated an item, I use -1 as the "missing data" value.
    
    This is an abstract class because the instantiation differs depending on where you get the  
    data from (a CSV file, a connection to the SWARM database + a question ID, etc.).
    """ 
    __metaclass__ = ABCMeta
    
    def get_missing_subset(self):
        """Return a dictionary of the indices of answers that have been scored and answers that  
        haven't (and raters that have provided scores, and raters that havent).
        Used in a preprocessing step in user_preferences(), to get a subset of the score matrix 
        where unrated items have been dropped (and useless raters have been dropped).
        Retaining a list of indices of the dropped items means we can still create an overall  
        ordering of all items (rated and unrated).
        """
        
        subset = {'answers':{'data':np.where(np.logical_not(np.all(self.matrix == -1, 
                                                                   axis=0)))[0], \
                             'missing':np.where(np.all(self.matrix == -1, axis=0))[0]}, \
                  'raters':{'data':np.where(np.logical_not(np.all(self.matrix == -1, 
                                                                  axis=1)))[0], \
                            'missing':np.where(np.all(self.matrix == -1, axis=1))[0]}}
        return subset
    
    def anca_rating(self):
        """modified version of user preference rankings, suggested by Anca Hanea.
        
        If a particular rater prefers Answer A to Answer B, that is counted as
        a win for Answer A. 
        
        The pairwise score difference for Answer A, relative to Answer B, is 
        (# wins - # losses) / (# of raters who have rated both A and B).
        
        Answer A's *overall* score is the sum of its pairwise differences with
        all the other answers.
        
        This means some answers get negative scores, while answers that recieve 
        no ratings at all would score better (0)! To avoid this, I shift up the
        scores so every answer that has a rating is at least one point higher 
        than the unrated answers.
        """
        
        # start by excluding the answers that have no ratings and the raters
        # that didn't submit any ratings
        subset = {'answers':{'data':np.where(np.logical_not(np.all(self.matrix == -1, 
                                                                   axis=0)))[0], \
                             'missing':np.where(np.all(self.matrix == -1, axis=0))[0]}, \
                  'raters':{'data':np.where(np.logical_not(np.all(self.matrix == -1, 
                                                                  axis=1)))[0], \
                            'missing':np.where(np.all(self.matrix == -1, axis=1))[0]}}
        
        m = len(subset["answers"]["data"])
        
        # if no answers have been rated, or no raters have rated anything
        if (m < 1 or len(self.subset["raters"]["data"]) < 1) or np.all(self.matrix == -1):
            return np.zeros((self.matrix.shape[1]))
        
        score_mat = self.matrix[subset['raters']['data'][:,None], 
                                subset['answers']['data']]
        
        # create and fill the matrix of pairwise differences
        kmat = np.zeros((m,m))
        
        for i in range(m):
            for j in range(i+1, m):
                
                # find the rows where both answers were rated
                both_rated = np.all((score_mat[:,[i,j]] > 0), axis=1)
                
                if np.any(both_rated):
                    num_pairs = sum(both_rated)
                    both_rated = np.where(both_rated)
                    
                    wins = np.sum(score_mat[both_rated, i] > score_mat[both_rated, j]) 
                    losses = np.sum(score_mat[both_rated, j] > score_mat[both_rated, i]) 
                    kmat[i,j] = (wins - losses) / num_pairs
                    kmat[j,i] = (losses - wins) / num_pairs
        
        # sum the pairwise differences to get overall scores
        rvec = np.sum(kmat,axis=1)/m
        
        # normalize, so everything that has received a rating is at least one point higher 
        # than unrated items
        rvec = rvec - np.min(rvec - 1)
        
        # Insert the rated items into a vector containing scores for all items
        all_rvec = np.zeros((len(self.answer_ids)))
        for i in range(len(rvec)):
            all_rvec[subset["answers"]["data"][i]] = rvec[i]
            
        return all_rvec
    
    def lizzie_rating(self):
        num_ratings = np.sum(self.matrix >= 0, axis = 0)
        anca_vec = self.anca_rating()
        
        test_tau, test_p_value = stats.kendalltau(anca_vec, num_ratings)
        
        # Conditions to exclude:
        # If either vector is all the same value (e.g. [2,2,2]), the test returns nan.
        # If the test is not statistically significant (because of, e.g., small number 
        # of answers or ratings), we don't want to add a bonus.
        # If the correlation is significant but negative I assume it is a false positive.
        
        if not np.isnan(test_tau) and (test_p_value < 0.05 and test_tau > 0):
            # start by normalizing the score vector so that values are between 0 and 1
            svec = anca_vec - np.min(anca_vec)
            svec = svec / float(np.max(svec))
            
            # normalize the count of ratings per answer so values are between 0 and 1
            rvec = num_ratings - np.min(num_ratings)
            rvec = rvec / float(np.max(rvec))

            # Add a bonus for number of ratings to the score vector. The size 
            # of the bonus is weighted by the correlation between the two vectors.
            wvec = svec + (rvec * test_tau)
            
        else:
            wvec = anca_vec
        
        # reweight so scores look more intuitive 
        # (note that this doesn't change the ordering!)
        wvec = wvec - np.min(wvec) + 0.15 * np.mean(wvec)
        wvec = (wvec / (np.max(wvec) + 0.15 * np.mean(wvec)))
            
        return wvec
        
    def mean_rating(self):
        """Returns the mean score of each item (i.e. the status quo of the platform)"""
        
        def nonmissing_mean(arr):
            if np.sum(arr >= 0) > 0:
                return np.mean(arr[np.where(arr >= 0)])
            else:
                return 0.0
        
        return np.apply_along_axis(nonmissing_mean, axis=0, arr=self.matrix)
    
    

## Define synthetic score matrix class


In [286]:
import scipy.stats as stats
from scipy.special import expit, logit

class ScoreMatrixSynthetic(AbstractScoreMatrix):
    """Generates a rater-by-answer matrix of scores for answers to the question.
    Note that -1 indicates missing data.
    
    Parameters:
        n: number of users (though not all of them have to contribute)
        rnoise: scaling factor, influences amount of noise in the ratings
        qnoise: RdR suggested this. Because the same people are submitting the answers
            and rating them, we expect a +ve correlation between the skill of the rater and
            the quality of that user's answer. When qnoise is larger, that correlation is
            smaller.
        punrated: the probability that a user will fail to rate an item
        pslacker: the probability that a user will fail to rate any items
        plurker: the probability that a user will fail to submit an answer
        toprated: if True, the user rates the answers they consider best. If False, they 
            rate a random selection of the answers.
        enoise: each rater has an "easiness" bias, i.e. how harsh or easy a rater they are.
            This is drawn from a Gaussian with mean zero, and enoise is its standard dev.
        diag: if True, users are allowed to rate their own answers; if False (the default)
             they can't.
             
    Note that the overall probabilities are different due to interactions between the 
    parameters. E.g. if pslacker=1, then none of the items will be rated, even if punrated=0.
    These are best considered as tuning parameters. Look at the overall number of ratings in
    the output when interpreting results.
    """ 
    
    def __init__(self, n=30, rnoise=0.5, qnoise=0.05, punrated=0.2, 
                 pslacker=.1, plurker=.2, enoise=.5, toprated=True, diag=False, weights=[]):
        self.matrix = self.get_score_matrix(n, rnoise, qnoise, punrated, pslacker, plurker, enoise, toprated, diag)
        self.answer_ids = range(self.matrix.shape[1])
        self.rater_ids = range(self.matrix.shape[0])
        self.subset = self.get_missing_subset()
        
        self.anca = self.anca_rating()
        self.lizzie = self.lizzie_rating()        
            
        
    def get_score_matrix(self, n=30, rnoise=0.5, qnoise=0.05, punrated=0.2, 
                         pslacker=.1, plurker=.2, enoise=.5, toprated=True, diag=False):
        """produces user-by-answer score matrix (as a 2D Numpy array)"""
        
        # draw from a truncated Gaussian distribution.
        # Note that this is a rejection sampling method, will be VERY SLOW if
        # sigma is large relative to the distance between lower and upper.
        def trunc_gauss(mu=0.5, sigma=1, lower=0, upper=1):
            if mu <= lower or mu >= upper:
                raise ValueError('mu outside bounds', mu, lower, upper)
            
            y = -1
            while y <= lower or y >= upper:
                y = np.random.normal(loc=mu,scale=sigma)
            return y
        
        trunc_gauss_vec = np.vectorize(trunc_gauss)
        
        # draw ability uniformly between 0 and 1
        ability = np.random.random(n)
        # draw each rater's "easiness" value from a gaussian
        easiness = np.random.normal(loc=0, scale=enoise, size=n)
        
        # add (truncated) Gaussian noise to get quality of answers
        quality = trunc_gauss_vec(ability, sigma=qnoise)
        
        # set up ratings matrix
        A = np.empty((n,n), dtype=float)
        for i in range(n):
            # draw all ratings
            ratings_i = trunc_gauss_vec(quality, sigma = (1-ability[i])*rnoise)
            # add easiness bias for this rater
            ratings_i = expit(logit(ratings_i) + easiness[i])
            A[i,:] = ratings_i
            
            # unrated: draw number of unrated answers
            ki = np.random.binomial(n, punrated)
            if toprated:
                # see that rater's top-rated answers
                Aki = np.argsort(A[i,:])[range(ki)]
            else: 
                # see ratings for a random set of answers
                Aki = np.random.choice(n, ki, replace=False)
            # set unrated elements to -1 (missing)
            if len(Aki) > 0:
                A[i, Aki] = -1

        # can't rate your own answers
        if not diag:
            np.fill_diagonal(A,-1)

        # "slackers" are people who don't rate answers
        # pick slackers with probability p, remove their rows from the matrix
        kslack = np.random.binomial(n, pslacker)
        slack = np.random.choice(n, kslack, replace=False)
        A = np.delete(A, slack, 0)

        # "lurkers" are people who don't write answers
        # pick lurkers with probability p, remove their columns from the matrix
        klurk = np.random.binomial(n, plurker)
        lurk = np.random.choice(n, klurk, replace=False)
        A = np.delete(A, lurk, 1)
        self.quality = np.delete(quality, lurk)
        
        return A
        

## ScoreMatrix (given data)

In [287]:
import scipy.stats as stats
from scipy.special import expit, logit

class ScoreMatrix(AbstractScoreMatrix):
    """Takes an numpy array containing a rater-by-answer matrix of scores, and 
    turns it into a ScoreMatrix object with all the rating aggregation functions.
    Note that -1 indicates missing data.
    """ 
    
    def __init__(self, arr):
        self.matrix = arr
        self.answer_ids = range(self.matrix.shape[1])
        self.rater_ids = range(self.matrix.shape[0])
        self.subset = self.get_missing_subset()
        
        self.anca = self.anca_rating()
        self.lizzie = self.lizzie_rating()         


In [288]:
problematic_case = np.array([[-1.        ,  0.69598663],
       [ 0.56882739, -1.        ],
       [ 0.70957079,  0.08803212],
       [ 0.5829268 ,  0.51257361]])

In [289]:
a = np.array([[.7, .57, .4],
              [.1, .7, .6],
              [.2, .5, .6]])


test_sm = ScoreMatrix(problematic_case)
test_sm.lizzie


array([ 0.91846298,  0.16869728])

In [291]:
test_sm = ScoreMatrixSynthetic(n=1)

mean_diffs = []
lizzie_diffs = []

print(test_sm.matrix)

print(np.round(test_sm.anca_rating() * 100.0))
print(np.round(test_sm.lizzie_rating() * 100.0))
print(np.round(test_sm.mean_rating() * 100))
print(np.round(test_sm.quality * 100))

[[-1.]]
[ 0.]
[ nan]
[ 0.]
[ 62.]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:140: RuntimeWarning: invalid value encountered in true_divide


In [249]:
print(np.mean(lizzie_diffs))
print(np.mean(mean_diffs))

0.244865235818
0.354530609104


In [250]:
len(lizzie_diffs)

704

In [199]:
test_sm.lizzie

array([ 0.560165  ,  0.39525403,  0.32496408,  0.71184185,  0.87376629,
        0.15022469,  0.67782464,  0.25320031,  0.65055905,  0.92556927,
        0.83546673,  0.31114461,  0.57951105,  0.07508036,  0.11850066])

In [88]:
if not (np.isnan(test_rho)):
    print(test_rho)
else:
    print("bye")

bye


In [89]:
b = ScoreMatrixSynthetic(n=4)
b.matrix

array([[-1.        ,  0.21978449,  0.1863181 ],
       [-1.        , -1.        ,  0.41789005],
       [ 0.0605736 ,  0.27369676,  0.14095515],
       [ 0.37586468,  0.14274267, -1.        ]])

In [29]:
test_sm = ScoreMatrixSynthetic(n=10, punrated = .8)
anca_vec = test_sm.anca
anca_min = np.min(anca_vec[np.where(anca_vec > 0.0)])
anca_vec, anca_min

(array([ 0.  ,  1.25,  0.  ,  0.  ,  1.  ,  1.  ,  1.75,  0.  ]), 1.0)

In [ ]:
import pandas as pd

## Plan for analysis:

* In worst case (high noise, few ratings, etc.), and in best case,
    * Which methods get the top answer right?
    * Which methods have highest Spearman's rho?
* What effect does setting topn=True have?
* What effect does the easiness bias have?

## Plot results

In [103]:
import matplotlib.pyplot as plt
%matplotlib inline

# Appendix: notes to self

## User preference ranking

In Chapter 10 of _Who's #1?_, Langville & Meyer describe a ranking method for _products_ (analogous to answers on SWARM) that have been rated by users according to a star rating scale - that is, given an integer score between 1 and 5. 

### Formulation:

If there are $m$ users and $n$ products, the ratings can be turned into an $m$-by-$n$ user-by-product matrix. 

$$\mathbf{A} =
 \begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{m,1} & a_{m,2} & \cdots & a_{m,n}
 \end{pmatrix}$$

For example:

|  | $p_1$ | $p_2$ | $p_3$ | $p_4$ | 
|---|---|---|---|---|
| $u_1$ |  3 |   |   |  5 | 
| $u_2$ |   | 2  | 2  |  4 | 
| $u_3$ |  3 |   |  3 |   | 
| $u_4$ |  1 | 4  |   |  5 | 
| $u_5$ |   |  5 |  2 |   | 
| $u_6$ |   |   |  1 |  3 | | 

Note that the ratings are _sparse_ - users do not have to rate all products.

Langville & Meyer begin (page 118) by defining an $n$-b-$n$ skew-symmetric matrix, $\mathbf{K}$, which holds the pairwise differences between the the products:

$$\mathbf{K} =
 \begin{pmatrix}
  0 & k_{1,2} & \cdots & k_{1,n} \\
  k_{2,1} & 0 & \cdots & k_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  k_{n,1} & k_{n,2} & \cdots & 0
 \end{pmatrix}$$

Then (page 128) they define the $k_{i,j}$ terms for this application:

$$k_{i,j} = -k_{j,i} = \begin{cases}
\frac{1}{n_{i,j}} \sum_{u \in U_i \cap U_j} a_{u,i} - a_{u,j} & \mbox{ if } n_{i,j} \neq 0\\
0 & \mbox{ if } n_{i,j} = 0
\end{cases}$$

where $U_i$ is the set of users who have rated product $p_i$, so $\{u: u \in U_i \cap U_j\}$ is the set of users who have rated both products $p_i$ and $p_j$, and $n_{i,j}$ is the number of users in $\{u: u \in U_i \cap U_j\}$.

Once we have $\mathbf{K}$, finding the overall rating vector $\mathbf{r}$ is simple. Simply take the row means - that is,

$$\mathbf{r} = \frac{\mathbf{Ke}}{n}$$

where $\mathbf{e}$ is a vector of ones.

### Properties

 - User preference ranking is based on pairwise comparisons between items.
 - It automatically takes into account that some people are "easy graders" - the absolute values of their ratings doesn't matter, only the _difference_ in the scores between pairs of items they have rated.
 - Users must rate at least two items for their ratings to affect the ranking.
 - Users influence pairwise comparisons whenever they rate both items in the pair. Call the "power" of a user the number of distinct pairs of items they have rated. If the user has rated $k$ items, their power is $\frac{k^2 - k}{2}$.
 - User preferences are vulnerable to users voting strategically like so: give an extremely high score to _one_ item, and extremely low scores to several other items. 